<a href="https://colab.research.google.com/github/varghesesanya/Accent-Conversion/blob/master/AITutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
!pip install langchain langchain-mistralai langchain_community mistralai==0.4.2
!pip install helper
!pip install -qU langchain-mistralai
!pip install python-dotenv
!pip install PyPDF2
!pip install langchain
!pip install sentence-transformers
!pip install -qU langchain-text-splitters



In [68]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
import getpass
import PyPDF2
import os
from dotenv import load_dotenv
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings


In [52]:
# Load environment variables from the .env file
load_dotenv('/content/drive/MyDrive/Computer Vision/AI Tutor Bonus Project/.env')

# Retrieve the API key
mistral_api_key = os.getenv("MISTRAL_API_KEY")

# Check if the API key is loaded correctly
if not mistral_api_key:
    raise ValueError("Mistral API key not found. Please check your .env file.")

# Set headers for the request
headers = {
    "Authorization": f"Bearer {mistral_api_key}"
}


In [53]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [54]:
pdf_path = "/content/drive/MyDrive/Computer Vision/AI Tutor Bonus Project/Computer_Vision_Richard_Szeliski-Chapter1.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
documents = [Document(page_content=pdf_text)]


In [76]:

# Split text into chunks

md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=60, chunk_overlap=0
)

markdown_text = """
# Title

This is a sample markdown text that will be split into chunks.
"""

md_docs = md_splitter.create_documents([pdf_text])
md_docs

text_splitter = RecursiveCharacterTextSplitter()
split_texts = text_splitter.split_text(pdf_text)  # Split directly into chunks of text

embeddings = HuggingFaceEmbeddings()

# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

vectorstore = InMemoryVectorStore.from_texts(
    split_texts,
    embedding=embeddings
)


# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# Show the retrieved document's content
if retrieved_documents:
    print(retrieved_documents[0].page_content)
else:
    print("No relevant documents found.")


<ipython-input-76-2ec0e7f36f8e>:21: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ure1.7d), along with the related 21/2-D sketch ideas of Marr (1982 ). This approach is again
seeing a bit of a revival in the work of Tappen, Freeman, and Adelson (2005 ).
More quantitative approaches to computer vision were also developed at the time, in-
cluding the ﬁrst of many feature-based stereo correspondence algorithms (Figure 1.7e) (Dev
1974 ;Marr and Poggio 1976 ;Moravec 1977 ;Marr and Poggio 1979 ;Mayhew and Frisby
1981 ;Baker 1982 ;Barnard and Fischler 1982 ;Ohta and Kanade 1985 ;Grimson 1985 ;Pol-
lard, Mayhew, and Frisby 1985 ;Prazdny 1985 ) and intensity-based optical ﬂow algorithms
7In robotics and computer animation, these linked-part graphs are often called kinematic chains .12 1 Introduction
(Figure 1.7f) (Horn and Schunck 1981 ;Huang 1981 ;Lucas and Kanade 1981 ;Nagel 1986 ).
The early work in simultaneously recovering 3D structure and camera motion (see Chapter 7)
also began around this time ( Ullman 1979 ;Longuet-Higgins 1981 ).
A lot of the philosophy of how visi

In [77]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

# Assuming vectorstore is already created and split_texts is the list of text chunks
# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Define the question you want to ask
query = "What are Vanishing Points"

# Retrieve the most similar text chunks from the vectorstore
retrieved_documents = retriever.invoke(query)

# Show the retrieved document's content
if retrieved_documents:
    print(retrieved_documents[0].page_content)
else:
    print("No relevant documents found.")



ing (see Section 14.1.1 );
•Visual authentication: automatically logging family members onto your home com-
puter as they sit down in front of the webcam (see Section 14.2).
The great thing about these applications is that they are already familiar to most students;
they are, at least, technologies that students can immediately appreciate and use with their
own personal media. Since computer vision is a challenging topic, given the wide range
of mathematics being covered4and the intrinsically difﬁcult nature of the problems being
solved, having fun and relevant problems to work on can be highly motivating and inspiring.
The other major reason why this book has a strong focus on applications is that they can
be used to formulate andconstrain the potentially open-ended problems endemic in vision.
For example, if someone comes to me and asks for a good edge detector, my ﬁrst question is
usually to ask why? What kind of problem are they trying to solve and why do they believe
that edge det